In [3]:
# %tensorflow_version 2.x
import tensorflow as tf

In [90]:
from tensorflow.keras.datasets import boston_housing
from tensorflow import keras

(X_train, y_train), (X_test, y_test) = boston_housing.load_data()

inputs = keras.Input(shape=(13,))  # each data has 13 features.
x = keras.layers.Dense(4, activation='relu')(inputs)
y = keras.layers.Dense(4, activation='relu')(x)
outputs = keras.layers.Dense(1, activation='linear')(y) # sigmoid -> linear :  sigmoid is for 0 or 1. 회귀이므로 linear나 None이 적당함. 

model = keras.Model(inputs, outputs)

In [92]:
X_train.shape

(404, 13)

### Custom Loss Function
Huber loss is not defined as part of the keras API. So we can implement it ourselves. It is good for situations where we don't want to penalize large differences as much as Mean Squared Error but more than Mean Absolute Error. (이 손실 함수는 예측값과 실제값 간의 큰 차이에 대해 MSE보다는 덜 민감하게, 그러나 MAE보다는 더 민감하게 반응하길 원할 때 유용합니다.)

#### 대표적인 손실함수
| 함수                            | 특징                               | 민감도              |
| ----------------------------- | -------------------------------- | ---------------- |
| **MSE** (Mean Squared Error)  | 오차를 **제곱**해서 평균                  | **큰 오차에 매우 민감**함 |
| **MAE** (Mean Absolute Error) | 오차의 **절댓값**을 평균                  | **큰 오차에 덜 민감**함  |
| **Huber Loss**                | **작은 오차는 MSE처럼, 큰 오차는 MAE처럼** 처리 | **중간 민감도**       |

#### Huber Loss는 언제 쓰나?
- 이상치(outlier) 가 있는 데이터셋에서 자주 사용합니다.
- MSE는 이상치가 크면 너무 과도하게 영향을 받습니다.
- MAE는 이상치에 둔감하지만, 학습이 느려질 수 있습니다.
- Huber Loss는 이 둘 사이의 절충안입니다.

In [95]:
def huber_fn(y_true, y_pred):
    error = y_true - y_pred
    is_small_error = tf.abs(error) < 1
    squared_loss = tf.square(error) / 2
    linear_loss  = tf.abs(error) - 0.5
    return tf.where(is_small_error, squared_loss, linear_loss)

### huber 함수 코드에 대해 설명하면, 
- error 계산
- error가 1보다 작으면 True, 아니면 False를 리턴
- mse를 계산
- mae를 계산
- is_small_error면 squared_loss를 아니면 linear_loss를 리턴

#### mse를 왜 2로 나누나?
| 항목                    | 설명                                    |
| --------------------- | ------------------------------------- |
| **왜 2로 나누나?**         | 미분 시 2가 생기므로, 처음부터 나눠서 미분을 간단히 만들기 위해 |
| **정식 MSE에 1/2 포함되나?** | 아니요. 그건 수학적으로 간편한 표현일 뿐               |
| **실제 손실값에는 영향 있나?**   | **없습니다.** 스케일만 바뀌며, 모델 성능은 동일합니다      |






In [98]:
# Now you can add your huber loss function directly into you model.compile
model.compile(loss=huber_fn, optimizer="adam")
model.fit(X_train, y_train, epochs=3)

Epoch 1/3
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 863us/step - loss: 78.1225
Epoch 2/3
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 923us/step - loss: 67.9978
Epoch 3/3
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 837us/step - loss: 61.1803


### Saving and Loading Models That Contain Custom Components
We need to pass the custom functions as a python dictionary to be able to reload functions from saved models

```python
keras.models.save_model(model, "./modelone.h5")
```  
=> WARNING:absl:You are saving your model as an HDF5 file via model.save() or keras.saving.save_model(model). This file format is considered legacy. We recommend using instead the native Keras format, e.g. model.save('my_model.keras') or keras.saving.save_model(model, 'my_model.keras').

```python
def huber_fn(y_true, y_pred):  
    error = y_true - y_pred  
    is_small_error = tf.abs(error) < 1  
    squared_loss = tf.square(error) / 2
    linear_loss  = tf.abs(error) - 0.5
    return tf.where(is_small_error, squared_loss, linear_loss)

loaded_model = keras.models.load_model("modelone.h5",
                                custom_objects={"huber_fn": huber_fn})
```
WARNING:absl:Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.

```python
loaded_model.fit(X_train, y_train, epochs=3)
```  
  
NotImplementedError: numpy() is only available when eager execution is enabled.

In [104]:
keras.models.save_model(model, "./modelone.keras")

In [106]:
# 커스텀 손실 함수 포함해서 모델 불러오기
loaded_model = keras.models.load_model("modelone.keras", custom_objects={"huber_fn": huber_fn})

# 모델 다시 컴파일 (metrics 포함)
loaded_model.compile(
    optimizer="adam",
    loss=huber_fn,
    metrics=["mae", "mse"]  # 원하는 metric 추가
)

| 상황    | 설명                                                 |
| ----- | -------------------------------------------------- |
| 경고 의미 | 저장된 옵티마이저(`rmsprop`)와 새 옵티마이저(`adam`)가 달라 충돌함      |
| 영향    | 없음 – 컴파일을 새로 하면 새로운 옵티마이저가 사용됨                     |
| 해결책   | `compile=False` 또는 `include_optimizer=False` 사용 가능 |

해결책 1
```python
model.save("modelone.keras", include_optimizer=False)
```
해결책 2
```python
loaded_model = keras.models.load_model("modelone.keras", compile=False)
loaded_model.compile(optimizer="adam", loss=huber_fn, metrics=["mae", "mse"])
```

In [109]:
loaded_model.fit(X_train, y_train, epochs=3)

Epoch 1/3
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 904us/step - loss: 53.9773 - mae: 54.4773 - mse: 3215.8723
Epoch 2/3
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 919us/step - loss: 46.8561 - mae: 47.3561 - mse: 2479.8958
Epoch 3/3
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 993us/step - loss: 41.3954 - mae: 41.8954 - mse: 1938.4512


### Custom Activation, Init, Regularizer, Contraints

In [112]:
def my_relu(z): # return value is just max of 0 and z
    return tf.maximum(0.,z)

def my_glorot_initializer(shape, dtype=tf.float32):
    stddev = tf.sqrt(2. / (shape[0] + shape[1]))
    return tf.random.normal(shape, stddev=stddev, dtype=dtype)

# def my_l1_regularizer(weights):
#     return tf.reduce_sum(tf.abs(0.01 * weights))

# def my_positive_weights(weights): # return value is just tf.nn.relu(weights)
#     return tf.where(weights < 0., tf.zeros_like(weights), weights)

# Because of error for the next code, i replaced it
from tensorflow.keras.regularizers import Regularizer
from tensorflow.keras.constraints import Constraint

# custom regularizer
class MyL1Regularizer(Regularizer):
    def __init__(self, l1=0.01):
        self.l1 = l1
    def __call__(self, x):
        return self.l1 * tf.reduce_sum(tf.abs(x))
    def get_config(self):
        return {'l1': self.l1}

# custom constraint
class MyPositiveWeights(Constraint):
    def __call__(self, w):
        return tf.where(w < 0., tf.zeros_like(w), w)


1. my_relu
   - 역할: ReLU(Rectified Linear Unit) 활성화 함수의 간단한 구현입니다.
   - 설명: 입력 z가 0보다 크면 그대로 반환하고, 0보다 작으면 0을 반환합니다.
   - tf.maximum(0., z) 는 각 요소별로 0과 z 중 큰 값을 선택합니다.
   - ReLU는 신경망에서 비선형성을 추가하는 데 많이 사용됩니다.
2. my_glorot_initializer
   - 역할: Glorot (Xavier) 초기화 방식을 따라 가중치를 초기화하는 함수입니다.
   - 설명:
     - shape: 초기화할 텐서의 모양 (예: (입력 노드 수, 출력 노드 수)
     - Glorot 초기화는 층의 입력과 출력 노드 수를 고려해 분산(표준편차)를 조절하여 가중치를 초기화합니다.
     - stddev 계산식은 Glorot 초기화에서 권장하는 값으로, 평균 0, 표준편차 sqrt(2/(fan_in + fan_out))를 가진 정규분포에서 값을 뽑습니다.
     - tf.random.normal로 주어진 표준편차를 가지는 정규분포 난수를 생성합니다.
   - 이 초기화는 신경망 학습 초기에 기울기 소실/폭주 문제를 완화하는 데 도움을 줍니다.
3. my_l1_regularizer
   - 역할: L1 규제(regularization)를 구현한 함수입니다.
   - 설명:
     - weights 텐서에 대해 각 원소 절댓값의 합을 계산해 규제값으로 반환합니다.
     - 0.01은 규제 강도(hyperparameter)로, 규제 영향력을 조절합니다.
     - L1 규제는 가중치가 0으로 수렴하게 만들어 모델을 희소(sparse)하게 하는 효과가 있습니다.
   - 규제항은 모델 학습 시 손실에 더해져 과적합(overfitting)을 줄이는 역할을 합니다.
4. my_positive_weights
   - 역할: 가중치가 음수일 경우 0으로 만드는 함수입니다.
   - 설명:
     - weights 텐서 내 음수인 원소 위치에는 0을,
     - 음수가 아닌 위치에는 원래 값(weights)을 반환합니다.
     - tf.where(condition, x, y)는 조건 condition이 참일 때 x를, 거짓일 때 y를 반환합니다.
     - 이 함수는 가중치 값이 항상 0 이상이도록 강제하는 역할을 하며, tf.nn.relu(weights)와 동일한 동작입니다.
   
| 함수명                     | 역할              | 주요 특징                  |
| ----------------------- | --------------- | ---------------------- |
| `my_relu`               | ReLU 활성화 함수     | 0과 입력 중 큰 값 반환         |
| `my_glorot_initializer` | Glorot 초기화      | 입력, 출력 노드 수 기준 가중치 초기화 |
| `my_l1_regularizer`     | L1 규제           | 가중치 절댓값 합산, 과적합 방지     |
| `my_positive_weights`   | 가중치 음수값을 0으로 변경 | 가중치 비음수 제약 (ReLU와 동일)  |

In [115]:
from tensorflow.keras.datasets import boston_housing
from tensorflow import keras

(X_train, y_train), (X_test, y_test) = boston_housing.load_data()

inputs = keras.Input(shape=(13,))
x = keras.layers.Dense(4, activation=my_relu,
                           kernel_initializer=my_glorot_initializer,
                           kernel_regularizer=MyL1Regularizer(0.01),
                           kernel_constraint=MyPositiveWeights())(inputs)
y = keras.layers.Dense(4, activation=my_relu,
                           kernel_initializer=my_glorot_initializer,
                           kernel_regularizer=MyL1Regularizer(0.01),
                           kernel_constraint=MyPositiveWeights())(x)
outputs = keras.layers.Dense(1, activation='linear')(y)  #회귀이므로 sigmoid 대신에 linear 로 바꿈

model = keras.Model(inputs, outputs)

In [117]:
model.compile(loss="mean_squared_error", optimizer="adam")
model.fit(X_train, y_train, epochs=3)

Epoch 1/3
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 860us/step - loss: 5913.4033
Epoch 2/3
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 900us/step - loss: 3279.3328
Epoch 3/3
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 856us/step - loss: 1529.6990
